In [1]:
import openai
from open_ai_utils import simular_respuesta_generativa
from LLMstoDataBase import SQLAgent

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 

import os
from dotenv import load_dotenv
import warnings

# Ignorar todos los warnings
warnings.filterwarnings('ignore')

# Cargar variables de entorno desde .env
load_dotenv()

# Acceder a la API key
api_key = os.getenv("API_KEY")
openai.api_key = api_key

In [2]:
'''
¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas, \
devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas?
'''

'\n¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas, devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas?\n'

In [3]:
finalizar = False
primera_consulta = True

pagila_bbdd = {
    'bbdd_name':'pagila', 
    'user'     :'postgres', 
    'password' :'123', 
    'host'     :'localhost', 
    'port'     :'5432'
}

# inicializamos el agente
agent = SQLAgent(**pagila_bbdd)

# entramos en la conversación-chat
while finalizar == False:

    conversacion = True
    nueva_consulta = True

    ### podria hacer una funcion para reconocer a que base de datos se quiere conectar el usuario

    # el usuario hace la consulta
    simular_respuesta_generativa('AGENTE:\nSoy tu agente experto en bases de datos. ¿En que te puedo ayudar hoy?\n\n')
    consulta_usuario = str(input())

    while conversacion: 
        
        if nueva_consulta== True:
            # printeamos la consulta del usuario
            simular_respuesta_generativa(f'\nPROMPT_USUARIO_{agent.historico.contador_interacciones+1}:\n\n{consulta_usuario}\n\n')

            # le pedimos la consulta al agente y printeamos la tabla
            tabla_resultado, codigo_sql = agent.nlp_to_sql(
                    consulta_nlp=consulta_usuario, 
                    metadata_token_limit= 1000, 
                            )      

            simular_respuesta_generativa(f'\nAGENTE:\nAqui tienes el resultado de tu consulta:\n{tabla_resultado}\n\n')

            # generamos el informe de la consulta dandole al agente la tabla el codigo sql y la consulta del usuario
            simular_respuesta_generativa('\nAGENTE:\nMe dispongo a generar el informe de tu consulta...\n\n')
            informe = agent.informe_resultado(
                    consulta_usuario= consulta_usuario, 
                    tabla_texto=tabla_resultado, 
                    max_tokens_respuesta=1000, 
                    codigo_sql= codigo_sql )
            nueva_consulta= False
                
            # printeamos el informe
            simular_respuesta_generativa(f'\nAGENTE:\nAqui tienes el informe de tu consulta: \n{informe}\n\n')

        # continuamos chat sobre la lectura o nueva consulta
        simular_respuesta_generativa('\nAGENTE:\n¿Tienes alguna otra consulta?\n\n')
        consulta_usuario = str(input())
        conversacion, nueva_consulta = agent.continuar_conversando(
            usuario= consulta_usuario, 
            tabla_consulta_anterior=tabla_resultado,
            codigo_sql_ejecutado=codigo_sql
            )

        while conversacion == True and nueva_consulta== False:

            # printeamos la consulta del usuario sobre respuestas anteriores
            simular_respuesta_generativa(f'\nPROMPT_USUARIO_{agent.historico.contador_interacciones +1}:\n{consulta_usuario}\n\n')

            respuesta_agente = agent.pregunta_sobre_consulta_anterior(
                usuario= consulta_usuario, 
                tabla_consulta_anterior=tabla_resultado, 
                consulta_sql_anterior=codigo_sql
                )
            # printeamos las nuevas respuesta del sistema
            simular_respuesta_generativa(f'\nAGENTE:\nLa respuesta a tu consulta:\n{respuesta_agente}\n\n')

            # continuamos chat sobre la lectura o nueva consulta
            simular_respuesta_generativa('\nAGENTE:\n¿Tienes alguna otra consulta?\n\n')
            consulta_usuario = str(input())
            conversacion, nueva_consulta = agent.continuar_conversando(
                usuario= consulta_usuario, 
                codigo_sql_ejecutado=codigo_sql, 
                tabla_consulta_anterior=tabla_resultado, 
                max_tokens_historico= 1500
                )

    simular_respuesta_generativa('AGENTE:\nEstas seguro de que quieres cerrar el chat? (Y/N)\n\n')
    ultima_oportunidad = str(input())

    if ultima_oportunidad.lower() == 'y': 
        finalizar = True
        agent.close_connection()

simular_respuesta_generativa(f'\n\nHa sido un placer ayudarte. Hasta la próxima!!')

AGENTE:
Soy tu agente experto en bases de datos. ¿En que te puedo ayudar hoy?


PROMPT_USUARIO_1:

¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas, \ devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas?


Voy a asegurar que los metadatos de bbdd pagila esten disponibles...

El archivo ya existe en: ../data/nlp_to_sql/pagila_simple_metadata.pickle
El archivo ya existe en: ../data/nlp_to_sql/pagila_full_metadata.pickle

Proceso de almacenamiento de metadatos para pagila completado.


AGENTE:
El codigo sql que se ejecutará para responder tu consulta:

SELECT a.first_name, a.last_name, f.title, f.rental_rate
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
JOIN film f ON fa.film_id = f.film_id
WHERE a.actor_id IN (
    SELECT actor_id
    FROM film_actor
    GROUP BY actor_id
    ORDER BY COUNT(film_id) DESC
    LIMIT 1
);


AGENTE:
Aqui tienes el resultado de tu consulta:
|    | first_name   | last_

In [8]:
import pickle

ruta_chats = 'chats/'
    # guardamos la instancia del agente
# ruta_agente = os.path.join(ruta_chats, 'agenteSQL.pickle')
# with open(ruta_agente, 'wb') as guardar_agente:
#     pickle.dump(agent, guardar_agente)

    # guardamos el historico en un txt 
historico_completo = agent.historico_completo.historico
ruta_historico_chat = os.path.join(ruta_chats, 'historico.txt')
with open(ruta_historico_chat, 'w', encoding='utf-8') as archivo_historico:
    archivo_historico.write(historico_completo)


In [4]:
print(agent.historico.historico)

prompt_1 USUARIO:
¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas, \ devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas?

respuesta_informe_1 AGENTE:
El informe detalla las películas en las que ha trabajado Gina Degeneres, la actriz con la mayor cantidad de películas en la base de datos consultada. Las películas varían en precio de alquiler, con tres tarifas distintas: $0.99, $2.99 y $4.99.

Resumen de tarifas de alquiler para las películas de Gina Degeneres:

- **$0.99**: 9 películas
  - Ejemplos incluyen "Excitement Eve", "Gandhi Kwai" y "Indian Love".
- **$2.99**: 10 películas
  - Ejemplos incluyen "Bed Highball", "Chaplin License" y "Chariots Conspiracy".
- **$4.99**: 23 películas
  - Ejemplos incluyen "Calendar Gunfight", "Chamber Italian" y "Coldblooded Darling".

En total, Gina Degeneres ha trabajado en 42 películas con una distribución variada de tarifas de alquiler. La tarifa más común es de $4.99

In [5]:
print(agent.historico_completo.historico)

prompt_1 USUARIO:
¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas, \ devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas?

codigo SQL_1 AGENTE:
SELECT a.first_name, a.last_name, f.title, f.rental_rate
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
JOIN film f ON fa.film_id = f.film_id
WHERE a.actor_id IN (
    SELECT actor_id
    FROM film_actor
    GROUP BY actor_id
    ORDER BY COUNT(film_id) DESC
    LIMIT 1
);

tabla_1 AGENTE:
|    | first_name   | last_name   | title                   |   rental_rate |
|---:|:-------------|:------------|:------------------------|--------------:|
|  0 | Gina         | Degeneres   | Bed Highball            |          2.99 |
|  1 | Gina         | Degeneres   | Calendar Gunfight       |          4.99 |
|  2 | Gina         | Degeneres   | Chamber Italian         |          4.99 |
|  3 | Gina         | Degeneres   | Chaplin License         |          2.99 |
|  4

In [6]:
for key, value in agent.historico_completo.info_consultas_estructurada.items():
    print(f'consulta_{key}')
    for n in value:
        print(n)

consulta_1
¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas, \ devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas?
|    | first_name   | last_name   | title                   |   rental_rate |
|---:|:-------------|:------------|:------------------------|--------------:|
|  0 | Gina         | Degeneres   | Bed Highball            |          2.99 |
|  1 | Gina         | Degeneres   | Calendar Gunfight       |          4.99 |
|  2 | Gina         | Degeneres   | Chamber Italian         |          4.99 |
|  3 | Gina         | Degeneres   | Chaplin License         |          2.99 |
|  4 | Gina         | Degeneres   | Chariots Conspiracy     |          2.99 |
|  5 | Gina         | Degeneres   | Clueless Bucket         |          2.99 |
|  6 | Gina         | Degeneres   | Coldblooded Darling     |          4.99 |
|  7 | Gina         | Degeneres   | Coneheads Smoochy       |          4.99 |
|  8 | Gina         | Deg